__tasks:__
- Converting `bed12` annotations from [HERVd: the Human Endogenous RetroViruses Database](https://herv.img.cas.cz/) to `gtf`
- Build alignment index for STAR and Salmon algorithms.

In [13]:
import subprocess

https://github.com/CGATOxford/cgat/blob/master/CGAT/scripts/bed2gff.py

In [17]:
def convert_bed12_to_gtf(BED):
    GTF = BED.replace('.bed.','.gtf.')
    cm = f'zcat {BED} | cgat bed2gff --as-gtf - | gzip > {GTF}'
    subprocess.run(cm,shell=True)

In [12]:
path_to_files = '/data_gilbert/home/aarab/tools/HERVs/files'

### erv

In [19]:
convert_bed12_to_gtf(
    BED = f'{path_to_files}/package-entities-erv.bed.gz'
)

### line

In [20]:
convert_bed12_to_gtf(
    BED = f'{path_to_files}/package-entities-line.bed.gz'
)

### rc

In [21]:
convert_bed12_to_gtf(
    BED = f'{path_to_files}/package-entities-rc.bed.gz'
)

### retroposon

In [22]:
convert_bed12_to_gtf(
    BED = f'{path_to_files}/package-entities-retroposon.bed.gz'
)

### satellite

In [23]:
convert_bed12_to_gtf(
    BED = f'{path_to_files}/package-entities-satellite.bed.gz'
)

### scrna

In [24]:
convert_bed12_to_gtf(
    BED = f'{path_to_files}/package-entities-scrna.bed.gz'
)

### sine

In [25]:
convert_bed12_to_gtf(
    BED = f'{path_to_files}/package-entities-sine.bed.gz'
)

### snrna

In [26]:
convert_bed12_to_gtf(
    BED = f'{path_to_files}/package-entities-snrna.bed.gz'
)

### trna

In [27]:
convert_bed12_to_gtf(
    BED = f'{path_to_files}/package-entities-trna.bed.gz'
)

## ...

    Error in `colnames<-`(`*tmp*`, value = c(paste(samplenames, "input", sep = "-"), : attempt to set 'colnames' on an object with less than two dimensions

In [18]:
%%bash
for gtf in files/*gtf.gz; do
    gtf1=${gtf/.gtf.gz/.c.gtf.gz};
    
    zcat $gtf | grep -v "#" | \
    awk 'BEGIN {OFS="\t"} NR==1 {print "seqname", "source", "feature", "start", "end", "score", "strand", "frame", "attribute"} {print}' - | \
    gzip > $gtf1;
done

## bed2fa

In [5]:
# !zcat files/package-entities-erv.gff3.gz | head -n 40

In [7]:
# !zcat files/package-entities-erv.bed.gz | head -n 40

`cgat` env

In [28]:
def convert_bed12_to_fasta(BED,ref_fa):
    FASTA = BED.replace('.bed.gz','.bed.fa.gz')
    cm = f'zcat {BED} | bedtools getfasta -split -name+ -fi {ref_fa} -bed - | gzip > {FASTA}'
    subprocess.run(cm,shell=True)

In [31]:
path_to_files = '/data_gilbert/home/aarab/tools/HERVs/files'
ref_fa='/data_gilbert/home/aarab/genomes/hg38/gencode.v34/GRCh38.primary_assembly.genome.fa'

for loop ...

In [38]:
for name in ['erv','line','rc','retroposon','satellite','scrna','sine','snrna','trna']:
    convert_bed12_to_fasta(
        BED = f'{path_to_files}/package-entities-{name}.bed.gz',
        ref_fa=ref_fa
    )

## STAR Index

`alignment` env

https://hbctraining.github.io/Intro-to-rnaseq-hpc-O2/lessons/03_alignment.html

In [2]:
# %%bash
# FASTA=~/genomes/hg38/gencode.v34/GRCh38.primary_assembly.genome.fa

# zcat from-dac-project/package-entities-satellite.bed.gz | \
# bedtools getfasta -name+ -s -fi $FASTA -bed - -split | head

In [3]:
# !head from-dac-project/package-entities-erv.gtf

In [4]:
# !zcat files/package-entities-erv.gff3.gz | head -n 30

In [5]:
cat star_index.sh

gtf=$1
jobs=$2

genome=${gtf/.gtf.gz/}

echo ${genome}

gunzip $gtf
gtf=${gtf/.gz/}

mkdir ${genome}_star_index

STAR \
    --runThreadN ${jobs} \
    --runMode genomeGenerate \
    --genomeDir ${genome}_star_index \
    --limitGenomeGenerateRAM 400000000000 \
    --genomeFastaFiles /data_gilbert/home/aarab/genomes/hg38/gencode.v34/GRCh38.primary_assembly.genome.fa \
    --sjdbGTFfile ${gtf} \
    --sjdbOverhang 99 &> ${genome}.log

gzip $gtf


In [6]:
%%bash
FASTA='/data_gilbert/home/aarab/genomes/hg38/gencode.v34/GRCh38.primary_assembly.genome.fa'

cd files
for gtf in *.gtf.gz; do
    echo `date`
    echo $gtf
    bash ../star_index.sh $gtf $FASTA 36
    echo "Done!"
    echo ""
done
cd ../

Sat Aug 26 16:46:44 PDT 2023
package-entities-erv.gtf.gz
package-entities-erv
Done!

Sat Aug 26 17:12:12 PDT 2023
package-entities-line.gtf.gz
package-entities-line
Done!

Sat Aug 26 17:39:05 PDT 2023
package-entities-rc.gtf.gz
package-entities-rc
Done!

Sat Aug 26 18:04:20 PDT 2023
package-entities-retroposon.gtf.gz
package-entities-retroposon
Done!

Sat Aug 26 18:30:06 PDT 2023
package-entities-satellite.gtf.gz
package-entities-satellite
Done!

Sat Aug 26 18:55:37 PDT 2023
package-entities-scrna.gtf.gz
package-entities-scrna
Done!

Sat Aug 26 19:20:46 PDT 2023
package-entities-sine.gtf.gz
package-entities-sine
Done!

Sat Aug 26 19:46:20 PDT 2023
package-entities-snrna.gtf.gz
package-entities-snrna
Done!

Sat Aug 26 20:11:34 PDT 2023
package-entities-trna.gtf.gz
package-entities-trna
Done!



mkdir: cannot create directory ‘package-entities-erv_star_index’: File exists


## Salmon index

In [23]:
def build_salmon_index(FASTA):
    index=FASTA.replace('.fa.gz','_salmon_index')
    cm = f'salmon index -t {FASTA} -i {index}'    
    print(FASTA)
    
    subprocess.run(cm,shell=True)

In [24]:
for name in ['erv','line','rc','retroposon','satellite','scrna','sine','snrna','trna']:
    build_salmon_index(
        FASTA = f'{path_to_files}/package-entities-{name}.bed.fa.gz',
    )

/data_gilbert/home/aarab/tools/HERVs/files/package-entities-erv.bed.fa.gz
/data_gilbert/home/aarab/tools/HERVs/files/package-entities-line.bed.fa.gz
/data_gilbert/home/aarab/tools/HERVs/files/package-entities-rc.bed.fa.gz
/data_gilbert/home/aarab/tools/HERVs/files/package-entities-retroposon.bed.fa.gz
/data_gilbert/home/aarab/tools/HERVs/files/package-entities-satellite.bed.fa.gz
/data_gilbert/home/aarab/tools/HERVs/files/package-entities-scrna.bed.fa.gz
/data_gilbert/home/aarab/tools/HERVs/files/package-entities-sine.bed.fa.gz
/data_gilbert/home/aarab/tools/HERVs/files/package-entities-snrna.bed.fa.gz
/data_gilbert/home/aarab/tools/HERVs/files/package-entities-trna.bed.fa.gz


## 

In [25]:
!date

Sat Sep  2 22:22:16 PDT 2023
